In [1]:
Year = 2021
RootDrive = "C:\\Users\\rianne.hartman\\Documents\\Stage\\"
print('The python root is: '+RootDrive)
FormFileNaam = str(Year)+" - Zorgpunt Waasland, Cluster West_Stageaanvraag formulier.xlsx"
BackupPath="C:\\Users\\rianne.hartman\\OneDrive - Zorgpunt Waasland\\Backup\\"

The python root is: C:\Users\rianne.hartman\Documents\Stage\


In [2]:
ProcessDelta = False
ExcellFile = RootDrive+'Input\\'+FormFileNaam
xls_path = RootDrive+'OutPut\\Stage_Output_'
#xls_Delta_path = RootDrive+'OutPut\\Delta_Files\\Delta_Stage_Output_'
xls_scholen_Path = RootDrive+'OutPut\\Scholen\\'
CorrectieFile = RootDrive+'Stage-Goedkeuring-{0}.xlsx'.format(str(Year))

DBFilePath = RootDrive+"Database\\Dataset_"+str(Year)+".sqlite"
print('________________________________________________________________________________________________')
print('|--- De gebruikte programma settings zijn:')
print('| * Inputfile:'+ExcellFile)
print('| * Outputfiles : '+xls_path+'*.xlsx')
print('| * Database : '+DBFilePath)
print('| * School Files : '+xls_scholen_Path+'[School]\\[Contact]-ReportFile*.xlsx')
print('| * Correctie File : '+CorrectieFile)
print('| * Schooljaar : '+str(Year)+' - '+str(Year+1))
print('|--- Start Programma')
print('________________________________________________________________________________________________')


________________________________________________________________________________________________
|--- De gebruikte programma settings zijn:
| * Inputfile:C:\Users\rianne.hartman\Documents\Stage\Input\2021 - Zorgpunt Waasland, Cluster West_Stageaanvraag formulier.xlsx
| * Outputfiles : C:\Users\rianne.hartman\Documents\Stage\OutPut\Stage_Output_*.xlsx
| * Database : C:\Users\rianne.hartman\Documents\Stage\Database\Dataset_2021.sqlite
| * School Files : C:\Users\rianne.hartman\Documents\Stage\OutPut\Scholen\[School]\[Contact]-ReportFile*.xlsx
| * Correctie File : C:\Users\rianne.hartman\Documents\Stage\Stage-Goedkeuring-2021.xlsx
| * Schooljaar : 2021 - 2022
|--- Start Programma
________________________________________________________________________________________________


In [3]:
def multipleReplace(text, wordDict):
    for key in wordDict:
        text = text.replace(key, wordDict[key])
    return text

ReplaceDict = {
    'bachelor':'',
    'brugopleiding':'',
    'Artevelde Hogeschool Gent':'Artevelde',
    'Hogeschool Gent':'HOGent',
    'HO_Gent':'HOGent',
    '(in het kader van TWE)':'',
    'gent':'Gent',
}
CorrectDict = {
    '-':'_',
    ' ':'_',
    '__':'_',
    '()':''
}

In [4]:
import warnings
import pandas as pd
import xlsxwriter as xw
import sqlite3
import datetime
from datetime import date
import os
import re

In [5]:
db = sqlite3.connect(DBFilePath)
MaxID = 0

In [6]:
warnings.filterwarnings('ignore', 
                        '.*', 
                        UserWarning)

In [7]:
df = pd.read_excel(ExcellFile, index_col=0)
df.reset_index(level=0, inplace=True)
df = df.fillna(0)
try:
    df = df.drop(['Start time','Completion time','Email','Name'], axis=1)
except:
    df = df.drop(['Begintijd','Tijd van voltooien','E-mail','Naam'], axis=1)
print('|> Excel file ingelezen')

|> Excel file ingelezen


In [8]:
cf = pd.read_excel(CorrectieFile, index_col=0)
cf.to_sql('Schema_CorrectieFile', con=db, if_exists='replace')
print('|> Correctie file ingelezen')
dt_date = datetime.datetime.now()
BackupDate = dt_date.strftime('%Y-%m-%d')
BackupFile = BackupPath+BackupDate+'-CorrectieFile.xlsx'
cf.to_excel(BackupFile)
print('|> Correctie file backup gemaakt...')

|> Correctie file ingelezen
|> Correctie file backup gemaakt...


In [18]:
df_Data=df[['ID','Aanduiden keuze voor WZC',
 'Naam van de school of onderwijsintelling',
 'Naam aanvrager',
 'E-mail adres aanvrager',
 'Telefoonnummer aanvrager',
 'Naam student',
 'Studierichting/stage',
 'Studiejaar / Module',
 'Specialisatie',
 'Voorkeursafdeling',
 'Totaal aantal stage uren',
 'Totaal aantal stage dagen',
 'Aantal te presteren uren per stage dag']]
df_Data.to_sql('Schema_InputFile', con=db, if_exists='replace')
print('|-> Stagair data van Excel file naar de DB geschreven')

KeyError: "['Naam'] not in index"

In [ ]:
df_Dagen=df[['ID',
'1 ste Stagedag','2 de Stagedag','3 de Stagedag','4 de Stagedag','5 de Stagedag',
'6 de Stagedag','7 de Stagedag','8 ste Stagedag','9 de Stagedag','10 de Stagedag',
'11 de Stagedag','12 de Stagedag','13 de Stagedag','14 de Stagedag','15 de Stagedag',
'16 de Stagedag','17 de Stagedag','18 de Stagedag','19 de Stagedag','20 ste Stagedag',
'21 ste Stagedag','22 ste Stagedag','23 ste Stagedag','24 ste Stagedag','25 ste Stagedag',
'26 ste Stagedag','27 ste Stagedag','28 ste Stagedag','29 ste Stagedag','30 ste Stagedag',
'31 ste Stagedag','32 ste Stagedag','33 ste Stagedag','34 ste Stagedag','35 ste Stagedag',
'36 ste Stagedag','37 ste Stagedag','38 ste Stagedag','39 ste Stagedag','40 ste Stagedag',
'41 ste Stagedag','42 ste Stagedag','43 ste Stagedag','44 ste Stagedag','45 ste Stagedag',
'46 ste Stagedag','47 ste Stagedag','48 ste Stagedag','49 ste Stagedag','50 ste Stagedag']]
#df_Dagen.to_sql('Schema_DagenInput', con=db, if_exists='replace')

In [ ]:
EmptyFrame = False
#create a list of all the columns
columns = list(df_Dagen)
#create lists to hold headers & months
headers = []
days = []
#split columns list into headers and months
for col in columns:
    if col.endswith('Stagedag'):
        days.append(col)
    else:
        headers.append(col)

#columns
D_Dag=pd.melt(df_Dagen,id_vars=headers,value_vars=days,var_name='Comment',value_name='Date')
D_Dag = D_Dag[D_Dag.Date != 0]
D_Dag = D_Dag.drop(['Comment'], axis=1)
D_Dag['Dagen']=D_Dag['Date'].astype('datetime64[ns]')
del D_Dag['Date']
D_Dag['Dag']=D_Dag['Dagen'].dt.day
D_Dag['Maand']=D_Dag['Dagen'].dt.month
D_Dag['Jaar']=D_Dag['Dagen'].dt.year
D_Dag['IngeSchreven']=1

In [ ]:
D_Dag.to_sql('Schema_D_Dag', con=db, if_exists='replace')
print('|-> Stage dagen weggeschreven naar de DB')

In [ ]:
DB_Correctie_query = """
select 
    ID*1000000+Jaar*100+Maand as Key,
    ID,
    dag,
    Maand,
    Jaar,
    MAX(Ingeschreven) as IngeSchreven
from Schema_D_Dag
group by
    ID,
    dag,
    Maand,
    Jaar"""
D_Dag = pd.read_sql_query(DB_Correctie_query,db)
print('|-> Eventuele dubbels verwijderd uit de DB')

In [ ]:
def CreateMonthFrame(Inputdf,MaandNr,Jaar):
    Query = 'Maand=='+str(MaandNr)+' and Jaar=='+str(Jaar)
    df = Inputdf.query(Query)
    return df

In [ ]:

if EmptyFrame == False:    
    columns = ['ID',1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31]
    todays_date = datetime.datetime.now().date()
    index = pd.date_range(todays_date-datetime.timedelta(10), periods=10, freq='D')
    MaandResultaat = pd.DataFrame(index=index, columns=columns)
    for Maand in range (1,13):
        #Month = '00'+str(Maand).strip()
        #Month = Month[-2:]
        if Maand < 7:
            Jaar = str(Year+1)
        else:
            Jaar = str(Year)
        print('|--> Verwerken van de maandgegevens voor maand nr.: '+str(Maand)+'...')
        MonthFrame = CreateMonthFrame(D_Dag,Maand,Jaar)
        if MonthFrame.empty:
            print('|----> Nog geen gegevens deze maand, niets te doen.')
        else:
            DatumResultaat = pd.DataFrame([]).fillna(0)
            DatumResultaat = MonthFrame.pivot(index='ID', columns='Dag', values='IngeSchreven')
            DatumResultaat['Maand']=Maand
            DatumResultaat['Jaar']=Jaar
            DatumResultaat['ID']=DatumResultaat.index
            MaandResultaat = MaandResultaat.append(DatumResultaat).fillna(0)


In [ ]:
MaandResultaat=MaandResultaat.apply(pd.to_numeric)
print('|--> Dataframe genormaliseerd')

In [ ]:
if EmptyFrame == False:
    DataSet = pd.merge(df_Data, MaandResultaat,how='left', on=['ID'])
    DataSet = pd.merge(cf,DataSet,how='inner', on=['ID'])
    SchemaDataSet = 'Schema_Dataset_'+str(Year+1)
    if ProcessDelta:
        DataSet.to_sql(SchemaDataSet, con=db, if_exists='append')
    else:
        DataSet.to_sql(SchemaDataSet, con=db, if_exists='replace')
        print('|--> Dataframe opgeslagen in de database')
else:
    print('|--> Niets te doen...')

In [ ]:
###################################
# Query om zowel de totalen als de detail 
# Uit de dataset te halen
DB_Correctie_query = """;with StageSet as
(SELECT
	D."ID",
	D."Status",
	D."Aanduiden keuze voor WZC" as WCZ,
	trim(COALESCE(C.[Correctie Naam en Voornaam],D."Naam student")) as "Naam student",
	(trim(D."Naam van de school of onderwijsinstelling")) as School,
	D."Naam aanvrager" as Naam_aanvrager,
  	trim(D."E-mail adres aanvrager") as "Email_aanvrager",
  	D."Telefoonnummer aanvrager" as Tel_aanvrager,
  	D."Studierichting/stage" as Studierichting_stage,
  	D."Studiejaar / Module" as Studiejaar_Module,
  	D."Specialisatie",
  	D."Voorkeursafdeling",
  	D."Totaal aantal stage uren" as stage_uren,
  	D."Totaal aantal stage dagen" as stage_dagen,
  	D."Aantal te presteren uren per stage dag" as uren_per_dag,
  	D."Jaar",
	D."Maand",
	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."1" END "1",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."2" END "2",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."3" END "3",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."4" END "4",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."5" END "5",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."6" END "6",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."7" END "7",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."8" END "8",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."9" END "9",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."10" END "10",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."11" END "11",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."12" END "12",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."13" END "13",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."14" END "14",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."15" END "15",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."16" END "16",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."17" END "17",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."18" END "18",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."19" END "19",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."20" END "20",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."21" END "21",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."22" END "22",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."23" END "23",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."24" END "24",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."25" END "25",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."26" END "26",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."27" END "27",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."28" END "28",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."29" END "29",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."30" END "30",
  	CASE WHEN C.Status in ('Niet OK','Fout') THEN 0 ELSE D."31" END "31"
From "Schema_CorrectieFile" C
INNER JOIN """+SchemaDataSet+""" D on C.ID=D.ID
WHERE C.Status not in ('Niet OK','Fout')
AND C.ID > 0
AND NOT Jaar is null
AND Jaar between """+str(Year)+""" and """+str(Year+1)+"""
)
, Totalen as (
select * FROM StageSet
UNION ALL
select 
	999999 as ID,
	'Totalen' as "Status",
	WCZ,
	COUNT(DISTINCT("Naam student")) as "Aantal Stagairs",
	COUNT(DISTINCT("School")) as "Aantal Scholen",
	Null as Naam_aanvrager,
  	Null as "Email_aanvrager",
  	null as Tel_aanvrager,
  	Studierichting_stage as Studierichting_stage,
  	Null as Studiejaar_Module,
  	Null as "Specialisaties",
  	NULL as "Voorkeursafdeling",
  	Null as stage_uren,
  	Null as stage_dagen,
  	Null as uren_per_dag,
	"Jaar",
	"Maand",
	SUM("1") as "1",
	SUM("2") as "2",
	SUM("3") as "3",
	SUM("4") as "4",
	SUM("5") as "5",
	SUM("6") as "6",
	SUM("7") as "7",
	SUM("8") as "8",
	SUM("9") as "9",
	SUM("10") as "10",
	SUM("11") as "11",
	SUM("12") as "12",
	SUM("13") as "13",
	SUM("14") as "14",
	SUM("15") as "15",
	SUM("16") as "16",
	SUM("17") as "17",
	SUM("18") as "18",
	SUM("19") as "19",
	SUM("20") as "20",
	SUM("21") as "21",
	SUM("22") as "22",
	SUM("23") as "23",
	SUM("24") as "24",
	SUM("25") as "25",
	SUM("26") as "26",
	SUM("27") as "27",
	SUM("28") as "28",
	SUM("29") as "29",
	SUM("30") as "30",
	SUM("31") as "31"
from StageSet
WHERE NOT Jaar is null
AND Jaar between """+str(Year)+""" and """+str(Year+1)+"""
GROUP by WCZ,Studierichting_stage,"Maand","Jaar")

select * from Totalen
"""

if EmptyFrame == False:
    DataSet = pd.read_sql_query(DB_Correctie_query,db)
    DataSet.to_sql('Results_Dataset', con=db, if_exists='replace')
    DataSet = DataSet.apply(pd.to_numeric, downcast='integer', errors='ignore')
    print('|--> Stagedata beschikbaar om mee te werken...')
    
else:
    print('|--> Niets te doen...')


In [ ]:
############################################
# Creëer de Scholen DataSet
DB_Correctie_query = """SELECT DISTINCT(trim("Naam van de school of onderwijsinstelling")
) as School FROM """+SchemaDataSet
Scholen = pd.read_sql_query(DB_Correctie_query,db).replace(ReplaceDict, regex=True).replace(CorrectDict, regex=True)
if Scholen.empty:
    print('|--> GEEN Schooldata beschikbaar om mee te werken...')
else:
    print('|--> Schooldata beschikbaar om mee te werken...')

In [ ]:
############################################
#Close & Commit the Connection
try:
    db.commit()
    print('|--> Alle benodigde data in de DB opgeslagen')
    db.close()
    print('|--> DB Gesloten ')
except:
    print('|--> DB was al afgesloten')

In [ ]:
print('|--> Start met de verwerking van de datumgegevens')

In [ ]:
MaandenSet = pd.DataFrame({'ProcessID':[1,2,3,4,5,6,7,8,9,10,11,12],
                           'MaandID':[7,8,9,10,11,12,1,2,3,4,5,6],
                           'Maand_Jaar':['juli '+str(Year),'augustus '+str(Year),'september '+str(Year),'oktober '+str(Year),'november '+str(Year),'december '+str(Year),
                                        'januari '+str(Year+1),'februari '+str(Year+1),'maart '+str(Year+1),'april '+str(Year+1),
                                        'mei '+str(Year+1),'juni '+str(Year+1)]})
#MaandenSet = MaandenSet.apply(pd.to_numeric, downcast='integer', errors='ignore')
df1 = DataSet
df1.columns = [c.replace(' ', '_') for c in df1.columns]
df1.columns = [c.replace('-', '_') for c in df1.columns]
WCZs = df1.WCZ.unique()
print('|--> WCZ data beschikbaar om mee te werken...')

In [ ]:
Maanden = pd.to_numeric(DataSet['Maand'], downcast='integer',errors='ignore').unique()
Maanden = pd.DataFrame({'MaandID': Maanden[:,]})
Maanden = pd.merge(Maanden, MaandenSet,how='left', on='MaandID')
Maanden.set_index('ProcessID', inplace=True)
Maanden.sort_index(inplace=True)
print('|--> Maand/Jaar data beschikbaar om mee te werken...')

In [ ]:
DataSet = DataSet.replace({"School": ReplaceDict}, regex=True).replace({"School": CorrectDict}, regex=True)
DataSet['School']=DataSet['School'].str.lower()

In [ ]:
######################################
# Reorder the dataset to allign conform the XLS Sheet

DataSet = DataSet[['ID','Status','Naam_student','School','Studierichting_stage',\
                   'Studiejaar_Module','Specialisatie','stage_uren',\
                   '1','2','3','4','5','6','7','8','9','10',\
                   '11','12','13','14','15','16','17','18','19','20', \
                   '21','22','23','24','25','26','27','28','29','30','31',\
                   'Maand','WCZ','Naam_aanvrager','Email_aanvrager','Tel_aanvrager',\
                   'Voorkeursafdeling','stage_dagen','uren_per_dag','Jaar']]
print('|--> Data geconfigureerd zoals in de Excel gewenst is...')


In [ ]:
def ExcelMarkup(DataSet,MaandID,WZC):
    if output.empty:
        process = 0
        print('|------->   Geen rijen voor: '+Maand_Jaar)
    else:
        print('|------->   We verwerken nu data voor: '+Maand_Jaar)
        output.to_excel(writer,'%s' % SheetName)
        process = 1
        if process == 1:
            ######################################
            # get sheets to add the tables
            workbook  = writer.book
            worksheet_table_header = writer.sheets[SheetName]

            ######################################
            # the range in which the table is
            end_row = len(output.index)
            end_column = len(output.columns)
            cell_range = xw.utility.xl_range(0, 0, end_row, end_column)

            ######################################
            # This is optional when using a solid fill.
            Green_Format = workbook.add_format({'bg_color': 'green'})
            Green_Format = Green_Format.set_pattern(1)
                
            ######################################
            # The hack
            # Using the index in the Table
            output.reset_index(inplace=True)
            header = [{'header': di} for di in output.columns.tolist()]
            worksheet_table_header.add_table(cell_range,\
                                             {'header_row': True,\
                                              'first_column': True,\
                                              'autofilter': False,\
                                              'columns':header,\
                                              'style': 'Table Style Medium 9'})
                
            ######################################
            # This is optional when using a solid fill.
                
            White_Format = workbook.add_format({'color': '#FFFFFF'})
            White_Format.set_bold(True)
            White_Format.set_pattern(1)
            Green_Format = workbook.add_format({'bg_color': 'green'})
            LGreen_Format = workbook.add_format({'bg_color': '#FF99BB'})
            worksheet_table_header.set_column(0, 1, 0)
            worksheet_table_header.set_column(2, 2, 6)
            worksheet_table_header.set_column(3,3,17.4)
            worksheet_table_header.set_column(4, 5, 11)
            worksheet_table_header.set_column(6, 6, 8.2)
            worksheet_table_header.set_column(7, 7, 9.8)
            worksheet_table_header.set_column(8, 8, 4.4)
            worksheet_table_header.set_column(9, end_column-6, 2.5)
            worksheet_table_header.set_column(end_column-5,end_column-5,18)
            worksheet_table_header.set_column(end_column-4,end_column-4,22)
            worksheet_table_header.set_column(end_column-3,end_column-3,13)
            worksheet_table_header.set_column(end_column-2,end_column-2,18)
            worksheet_table_header.set_column(end_column-1,end_column,11)
            #worksheet_table_header.set_column(1, 1, None, Hidden_format)
            #worksheet_table_header.set_default_row(hide_unused_rows=True)
            worksheet_table_header.conditional_format('J2:AN800', {'type':\
                                                                   'cell',\
                                                                   'criteria': '>=',\
                                                                   'value':    1,\
                                                                   'format':   Green_Format})
            worksheet_table_header.conditional_format('J2:AN800', {'type':\
                                                                   'cell',\
                                                                   'criteria': '>=',\
                                                                   'value':    2,\
                                                                   'format':   White_Format})
            worksheet_table_header.conditional_format('C2:C800', {'type':\
                                                                  'cell',\
                                                                  'criteria': '==',\
                                                                  'value':    '"OK"',\
                                                                  'format':   LGreen_Format})

            ######################################
            # The Outline Levels per type.
            #LogiStiek_Format = worksheet_table_header.set_row(None, None, None, {'level': 1})
                
print('|--> Excel sheet formatting Function Loaded...')

In [ ]:
if EmptyFrame == False:
    print('|--> Start processing...')
    for WZC in WCZs:
        print('|--->   We werken nu voor : '+WZC)
        FileName  = xls_path+str(Year)+'-'+str(Year+1)+'_'+WZC+'.xlsx'
        Sheet = DataSet
        ######################################
        # Creating the file
        
        writer = pd.ExcelWriter(FileName, engine='xlsxwriter')

        for index, row in Maanden.iterrows():
            SheetName = str(row['Maand_Jaar']).replace(' ', '_')
            Maand_Jaar = row['Maand_Jaar']
            
            Maand_Jaar = str(Maand_Jaar)
            if Maand_Jaar == 'nan':
                print('|----->   Found Nan')
            else:
                MaandID = row['MaandID']
                #print('|----->   We verwerken nu data voor: '+Maand_Jaar)
                
                ######################################
                # Filter the data before writing to the Excell sheet
                output = DataSet.loc[DataSet['Maand'] == MaandID]
                output = output.loc[output['WCZ'] == WZC]
                output = output.sort_values(by=['Studierichting_stage','ID'])
                output = output.drop(['WCZ','Maand','Jaar'], axis=1)
                ######################################
                # Write the data to the Excell sheet
                ExcelMarkup(output,MaandID,WZC)
            
        ######################################
        # Closing the file
        writer.save()
        print('|---> File: '+FileName+' aangemaakt')
else:
    print('|----->   Niets te doen...')
print('________________________________________________________________________________________________')

In [ ]:
Scholen = Scholen.replace({"School": ReplaceDict}, regex=True).replace({"School": CorrectDict}, regex=True)
Scholen = Scholen.drop_duplicates()
Scholen['School']=Scholen['School'].str.lower()
Scholen

In [ ]:
xls_scholen_Path = RootDrive+'OutPut\\School'
if not os.path.exists(xls_scholen_Path):
    os.makedirs(xls_scholen_Path)
if EmptyFrame == False:
    print('|--> Start processing...')
    for index, school in Scholen.iterrows():
        SchoolIndex = school['School']
        SchoolData = DataSet.loc[DataSet['School'] == SchoolIndex]
        School = school['School'].strip()
        School = School.replace('-','_')\
            .replace(' ','_')\
            .replace('_bachelor','')\
            .replace('_brugopleiding','')\
            .replace('Hogeschool_Gent','HOGent')\
            .replace('HO_Gent','HOGent')\
            .replace('(in_het_kader_van_TWE)','')\
            .replace('Artevelde_HOGent','Artevelde')\
            .replace('__','_') 
        
        ############################################
        # Creëer de Contact DataSet
        Email_aanvrager = SchoolData.Email_aanvrager.map(lambda x:x).unique()
        Email_aanvrager = pd.DataFrame(Email_aanvrager)
        if Email_aanvrager.empty:
            print('|--->   Er is voor deze periode geen contact data beschikbaar voor de school: '+School+'...')
        else:
            print('|-->   We rapporteren nu voor : '+School)
            School_Path = xls_scholen_Path+'\\'+School
            Email_aanvrager.columns = ['Email_aanvrager']
            Email_aanvrager = Email_aanvrager[Email_aanvrager.Email_aanvrager.str.len().fillna(0)>1]
            Email_aanvrager = Email_aanvrager.Email_aanvrager.map(lambda x:x).unique()
        
            #print('|--->   Er is contact data beschikbaar om mee te werken...')
            for contact in Email_aanvrager:
                print('|---->   We creëren een rapport voor : "'+contact+'"')
                contactnaam = contact.strip().replace(' ', '_').replace('.', '_').replace('@', '_at_').replace('-', '_')
                contactnaam = re.sub('[^\w\-_\. ]', '_', contactnaam).replace('_at_', '@')
                if not os.path.exists(School_Path):
                  os.makedirs(School_Path)
                  print('|---> Created Folder: '+School_Path)
                School_File_Path = School_Path+'\\'+School.strip()+'_'
                FileName = School_File_Path+'_'+contactnaam+'.xlsx'
                # print('|---->   We creëren de file : "'+FileName+'"')
                writer = pd.ExcelWriter(FileName, engine='xlsxwriter')

                for index, row in Maanden.iterrows():
                    SheetName = str(row['Maand_Jaar']).replace(' ', '_')
                    Maand_Jaar = row['Maand_Jaar']
                    Maand_Jaar = str(Maand_Jaar)
                    if Maand_Jaar == 'nan':
                        print('|----->   Found Nan')
                    else:
                        MaandID = row['MaandID']
                        ######################################
                        # Write the data
                        output = SchoolData.loc[SchoolData['Email_aanvrager'] == contact]
                        output = output.loc[output['Maand'] == MaandID]
                        #output = output.drop(['Email_aanvrager'], axis=1)
                        output = output.drop(['Tel_aanvrager','Jaar'], axis=1)
                        
                        ######################################
                        # Write the data to the Excell sheet
                        ExcelMarkup(output,MaandID,WZC)
                ######################################
                # Closing the file
                writer.save()
                print('|---->   File : '+FileName+' aangemaakt')
print('________________________________________________________________________________________________')